In [102]:
import os
import pandas as pd

def find_top_values_in_csv(directory):
    result_list = []
    
    # Find all CSV files with 'roc' in the filename
    csv_files = [f for f in os.listdir(directory) if 'pr' in f.lower() and f.endswith('.csv')]
    
    for file in csv_files:
        file_path = os.path.join(directory, file)
        
        # Read the CSV and use the first column as the index for row names
        df = pd.read_csv(file_path, index_col=0)
        
        # Convert all values to numeric, forcing non-numeric values to NaN
        df = df.apply(pd.to_numeric, errors='coerce')
        
        # Flatten the DataFrame, drop NaN values, and get the top 3 largest values
        df_values = df.values.flatten()
        valid_values = df_values[~pd.isna(df_values)]  # Remove NaN values
        
        if len(valid_values) < 3:
            print(f"Not enough valid data in {file}")
            continue  # Skip files with fewer than 3 valid data points
        
        # Sort in descending order and get the indices of the top 3 largest values
        sorted_indices = valid_values.argsort()[::-1][:3]
        
        # Initialize placeholders for top 3 values, row names, and column names
        top_values = [None] * 3
        row_names = [None] * 3
        col_names = [None] * 3
        
        for i, idx in enumerate(sorted_indices):
            # Get the flat index of the valid value in the original flattened array
            flat_index = df_values.tolist().index(valid_values[idx])  # Find the actual index in original flattened data
            row, col = divmod(flat_index, df.shape[1])  # Convert flat index to row, col
            
            # Populate the top values, exact row names, and column names
            top_values[i] = df.iat[row, col]
            row_names[i] = df.index[row]  # Use the exact row name here
            col_names[i] = df.columns[col]  # Use the exact column name here
        
        # Append the result for the current file
        result_list.append({
            "File Name": file,
            "AUCPR 1": top_values[0],
            "Transformation 1": row_names[0],
            "Classifier 1": col_names[0],
            "AUCPR 2": top_values[1],
            "Transformation 2": row_names[1],
            "Classifier 2": col_names[1],
            "AUCPR 3": top_values[2],
            "Transformation 3": row_names[2],
            "Classifier 3": col_names[2]
        })
    
    # Convert results to DataFrame
    result_df = pd.DataFrame(result_list)
    
    # Count unique occurrences for each individual column
    row_name_counts_1 = result_df["Transformation 1"].value_counts()
    row_name_counts_2 = result_df["Transformation 2"].value_counts()
    row_name_counts_3 = result_df["Transformation 3"].value_counts()

    column_name_counts_1 = result_df["Classifier 1"].value_counts()
    column_name_counts_2 = result_df["Classifier 2"].value_counts()
    column_name_counts_3 = result_df["Classifier 3"].value_counts()

    # Overall unique count across all row name columns
    overall_row_name_counts = result_df[["Transformation 1", "Transformation 2", "Transformation 3"]].stack().value_counts()
    # Overall unique count across all column name columns
    overall_column_name_counts = result_df[["Classifier 1", "Classifier 2", "Classifier 3"]].stack().value_counts()

    # Combine all row name frequencies into one DataFrame
    row_name_frequencies = pd.DataFrame({
        "Transformation 1 Count": row_name_counts_1,
        "Transformation 2 Count": row_name_counts_2,
        "Transformation 3 Count": row_name_counts_3,
        "Overall Count": overall_row_name_counts
    }).fillna(0).astype(int)  # Fill NaN with 0 and convert to integers

    # Combine all column name frequencies into one DataFrame
    column_name_frequencies = pd.DataFrame({
        "Classifier 1 Count": column_name_counts_1,
        "Classifier 2 Count": column_name_counts_2,
        "Classifier 3 Count": column_name_counts_3,
        "Overall Count": overall_column_name_counts
    }).fillna(0).astype(int)  # Fill NaN with 0 and convert to integers

    return result_df, row_name_frequencies, column_name_frequencies

# Example usage:
root = "C:\\Users\\Shehroz.Khan\\OneDrive - Al-Arabia Educational Enterprises Company\\OCC - Categorical Data\\"
directory = root+"\\Output Files"  # Replace with the actual directory
summary_df, row_name_freq, col_name_freq = find_top_values_in_csv(directory)
print(summary_df)

# Save the summary to a CSV file if needed
directory = root+"\\Results"
summary_df.to_csv(directory+"\\top_summary_aucpr.csv", index=False)
# Save combined frequency dataframes
row_name_freq.to_csv(directory+"\\transformation_frequencies_aucpr.csv")
col_name_freq.to_csv(directory+"\\classifier_frequencies_aucpr.csv")


                   File Name   AUCPR 1 Transformation 1 Classifier 1  \
0      advert0_output_pr.csv  0.939124          OHE-MCA          LOF   
1      advert1_output_pr.csv  0.933968              OHE    Euclidean   
2    audiology_output_pr.csv  0.848953           CAT-FH     Canberra   
3      boxing1_output_pr.csv  0.874687          OHE-MCA         ISOF   
4      boxing2_output_pr.csv  0.786290           CAT-FH    Euclidean   
5       breast_output_pr.csv  0.966895              OHE     Canberra   
6          car_output_pr.csv  0.856722              OHE          LOF   
7        chess_output_pr.csv  0.856218              OHE    Euclidean   
8   conference_output_pr.csv  0.937317        CAT-FH-SF    Euclidean   
9     diabetes_output_pr.csv  0.832726           OHE-AE    Euclidean   
10        dmft_output_pr.csv  0.903273          OHE-MCA     Canberra   
11       fraud_output_pr.csv  0.949500           OHE-AE    Euclidean   
12   marketing_output_pr.csv  0.851151          OHE-MCA     Canb

In [210]:
import os
import pandas as pd
from collections import defaultdict

def find_best_transformations_top3(directory, output_summary, output_matrix):
    result_list = []
    
    # New structure: classifier -> Top1/Top2/Top3 -> transformation -> count
    classifier_rank_counts = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

    # Step 1: Process each file
    csv_files = [f for f in os.listdir(directory) if f.endswith('pr.csv')]

    for file in csv_files:
        file_path = os.path.join(directory, file)
        df = pd.read_csv(file_path, index_col=0)

        row_data = {'File Name': file}
        for clf in df.columns:
            top_transforms = df[clf].nlargest(3).index.tolist()

            row_data[f'{clf}_Top1'] = top_transforms[0] if len(top_transforms) > 0 else ''
            row_data[f'{clf}_Top2'] = top_transforms[1] if len(top_transforms) > 1 else ''
            row_data[f'{clf}_Top3'] = top_transforms[2] if len(top_transforms) > 2 else ''

            for i, tr in enumerate(top_transforms):
                rank = f'Top{i+1}'
                classifier_rank_counts[clf][rank][tr] += 1

        result_list.append(row_data)

    # Step 2: Create summary DataFrame
    summary_df = pd.DataFrame(result_list)

    # Step 3: Create the new count DataFrame format
    all_transformations = set()
    for clf in classifier_rank_counts:
        for rank in classifier_rank_counts[clf]:
            all_transformations.update(classifier_rank_counts[clf][rank].keys())

    # Create full matrix with transformations as rows and classifier_rank as columns
    matrix_data = []
    for tr in sorted(all_transformations):
        row = {'Transformation': tr}
        for clf in sorted(classifier_rank_counts.keys()):
            for rank in ['Top1', 'Top2', 'Top3']:
                col_name = f'{clf}_{rank}'
                count = classifier_rank_counts[clf][rank].get(tr, 0)
                row[col_name] = count
        matrix_data.append(row)

    count_df = pd.DataFrame(matrix_data)
    count_df = count_df.set_index('Transformation')

    # Step 4: Save outputs
    summary_path = os.path.join(directory, output_summary.lstrip(os.sep))
    matrix_path = os.path.join(directory, output_matrix.lstrip(os.sep))

    summary_df.to_csv(summary_path, index=False)
    count_df.to_csv(matrix_path)

    print(f"Saved summary to {summary_path}")
    print(f"Saved matrix to {matrix_path}")
    
    return summary_df, count_df


In [226]:
# Define the directory with the ROC files
roc_dir = "C:/Users/Shehroz.Khan/OneDrive - Al-Arabia Educational Enterprises Company/OCC - Categorical Data/OutPut Files"

# Define the base directory where you want results
results_dir = os.path.join(os.path.dirname(roc_dir), "Results")
os.makedirs(results_dir, exist_ok=True)  # Ensure it exists

# Define full output file paths
summary_path = os.path.join(results_dir, "summary_top3_transformations_aucpr.csv")
matrix_path = os.path.join(results_dir, "top3_transformation_counts_aucpr.csv")

find_best_transformations_top3(roc_dir, summary_path, matrix_path)


Saved summary to C:/Users/Shehroz.Khan/OneDrive - Al-Arabia Educational Enterprises Company/OCC - Categorical Data\Results\summary_top3_transformations_aucpr.csv
Saved matrix to C:/Users/Shehroz.Khan/OneDrive - Al-Arabia Educational Enterprises Company/OCC - Categorical Data\Results\top3_transformation_counts_aucpr.csv


(                   File Name Euclidean_Top1 Euclidean_Top2 Euclidean_Top3  \
 0      advert0_output_pr.csv        OHE-MCA            OHE         OHE-AE   
 1      advert1_output_pr.csv            OHE        OHE-MCA         CAT-SF   
 2    audiology_output_pr.csv         CAT-FH         OHE-AE            OHE   
 3      boxing1_output_pr.csv         CAT-FH      CAT-FH-SF        OHE-MCA   
 4      boxing2_output_pr.csv         CAT-FH      CAT-FH-SF            OHE   
 5       breast_output_pr.csv            OHE        OHE-MCA         CAT-FH   
 6          car_output_pr.csv      CAT-FH-SF         CAT-FH         CAT-SF   
 7        chess_output_pr.csv            OHE        OHE-MCA         OHE-AE   
 8   conference_output_pr.csv      CAT-FH-SF         OHE-SF         CAT-FH   
 9     diabetes_output_pr.csv         OHE-AE            OHE        OHE-MCA   
 10        dmft_output_pr.csv        OHE-MCA         CAT-SF         OHE-SF   
 11       fraud_output_pr.csv         OHE-AE      OHE-AE-SF     

In [228]:
#Convert .xlsx to latex
import pandas as pd

# Load your Excel file
df = pd.read_excel("C:\\Users\\Shehroz.Khan\\OneDrive - Al-Arabia Educational Enterprises Company\\OCC - Categorical Data\\Results\\Summarized Results - AUCPR.xlsx", sheet_name="Sheet1")

# Convert to LaTeX
latex_table = df.to_latex(index=False)

# Save to file
with open("C:\\Users\\Shehroz.Khan\\OneDrive - Al-Arabia Educational Enterprises Company\\OCC - Categorical Data\\Results\\Summarized Results - AUCPR.tex", "w") as f:
    f.write(latex_table)


In [248]:
#Convert .csv file to latex
import pandas as pd

# Load your CSV file
df = pd.read_csv("C:\\Users\\Shehroz.Khan\\OneDrive - Al-Arabia Educational Enterprises Company\\OCC - Categorical Data\\Results\\result_Features.csv")  # Replace with your actual file path

# Convert DataFrame to LaTeX with custom formatting
latex_table = df.to_latex(index=False, 
                          escape=False, 
                          column_format='|' + '|'.join(['l'] * len(df.columns)) + '|', 
                          header=True)

# Add \hline after every row
lines = latex_table.splitlines()
formatted_lines = []

for i, line in enumerate(lines):
    formatted_lines.append(line)
    # Add \hline after header and each data row
    if i == 2 or (i > 2 and i < len(lines) - 1):
        formatted_lines.append('\\hline')
# Save to .tex file
with open("C:\\Users\\Shehroz.Khan\\OneDrive - Al-Arabia Educational Enterprises Company\\OCC - Categorical Data\\Results\\result_Features.tex", "w") as f:
    f.write(latex_table)
